# Necessary Packages & Enviorments
* NCBI Datasets (v18.9.0)
    * https://github.com/ncbi/datasets
    * conda install -c conda-forge ncbi-datasets-cli
* AUGUSTUS (v3.5.0)
    * https://github.com/Gaius-Augustus/Augustus
    * conda install bioconda::augustus
* Prodigal (v2.6.3)
    * https://github.com/hyattpd/Prodigal
    * conda install bioconda::prodigal
* FragGeneScan (v1.32)
    * https://github.com/gaberoo/FragGeneScan
    * Compiled using GitHub instructions
* GFFCompare (v0.12.10)
    * https://github.com/gpertea/gffcompare
    * conda install bioconda::gffcompare
* subprocess 
    * import subprocess

NOTE: The below code assumes that when you run the spcified section, if you created a unique conda enviorment for that package, that enviorment is activated. 

# Download Data
* We will be using the datasets command to download all assemblies and .gff files for the following organisms and associated GenBank accession numbers. The file will be downloaded as .zip files, which will need to be unzipped (which can be accomplished using `unzip "zip_*"`). This will cause some messages as unzipping all files will lead to overriding of the README file. Type yes for all, this will then create the directory `ncbi_dataset` which will have a subfolder called `data` where each assembly and GFF file lives for the specified accession numbers. I perfered to move the data folder to the directory that I was doing my work in to make the path less confusing, but this is up to you. 

| Species                   | Accession #        | GC Content | AUGUSTUS Bacteria Reference                                               | Bacteria GC |
|----------------------------|--------------------|-------------|----------------------------------------------------------------------|--------------|
| Nanoarchaeum equitans      | GCA_000008085.1    | 31.5%       | Staphylococcus aureus (*s_aureus*)                                   | 32.7%        |
| Methanococcus maripaludis  | GCA_002945325.1    | 33.0%       | Staphylococcus aureus (*s_aureus*)                                   | 32.7%        |
| Sulfolobus solfataricus    | GCA_003852155.1    | 36.0%       | Thermoanaerobacter tengcongensis (*thermoanaerobacter_tengcongensis*)| 37.5%        |
| Ferroplasma acidiphilum    | GCA_002078355.1    | 36.5%       | Thermoanaerobacter tengcongensis (*thermoanaerobacter_tengcongensis*)| 37.5%        |
| Cuniculiplasma divulgatum  | GCA_900083515.1    | 37.5%       | Thermoanaerobacter tengcongensis (*thermoanaerobacter_tengcongensis*)| 37.5%        |
| Pyrococcus furiosus        | GCA_008245085.1    | 41.0%       | Streptococcus pneumoniae (*s_pneumoniae*)                            | 39.5%        |
| Ferroglobus placidus       | GCA_000025505.1    | 44%         | Streptococcus pneumoniae (*s_pneumoniae*)                            | 39.5%        |
| Thermococcus kodakarensis  | GCA_000009965.1    | 52%         | Escherichia coli K-12 (*E_coli_K12*)                                 | 51%          |
| Natronomonas pharaonis     | GCA_000026045.1    | 63%         | Burkholderia pseudomallei (*b_pseudomallei*)                         | 68.0%        |
| Halorubrum ezzemoulense    | GCA_004126515.1    | 66.5%       | Burkholderia pseudomallei (*b_pseudomallei*)                         | 68.0%        |


In [ ]:
import subprocess

download_list = ["GCA_000008085.1", 
                 "GCA_002945325.1", 
                 "GCA_003852155.1", 
                 "GCA_002078355.1", 
                 "GCA_900083515.1", 
                 "GCA_008245085.1", 
                 "GCA_000025505.1", 
                 "GCA_000009965.1",
                 "GCA_000026045.1", 
                 "GCA_004126515.1"]


for item in download_list:
    file_name = "zip_" + item
    print(file_name)
    command = f'datasets download genome accession {item} --include gff3,genome --filename {file_name}'
    print(command)
    subprocess.run(command, shell=True)


# Process Data
* The GFF files that we downloaded include information that we don't need (i.e they include CDS and gene information, where for the scope of this project we are only focusing on CDS), this means that we need to parse them in order to get only the information we are intrested in. This will be done using the `grep` command. 

In [ ]:
for item in download_list: #for each organism 
    data_path = "data/" + item + "/genomic.gff" #path to where original gff file is
    folder_path = "data/" + item + "/" #path to folder for output
    #print(data_path)
    command = f'grep "CDS" {data_path} > {folder_path}{item}_CDS.gff' #grab all the CDS and put them into a CDS.gff file
    #print(command)
    subprocess.run(command, shell=True) #running command in terminal
    add_line_command = f"sed -i '1i##gff-version 3' {folder_path}{item}_CDS.gff" #.gff files require this as the first line, so add it in
    #print(add_line_command)
    subprocess.run(add_line_command, shell=True)


# AUGUSTUS Analysis



AUGUSTUS was initally make for eukaryotic speices, but since then it has expanded to support some prokaryotic organisms. Unlike the other two programs, AUGUSTUS requires a model. There are two ways that I have ran AUGUSTUS. First I have run all genomes with the only archaea species for which it has a model: Sulfolobus solfataricus. AUGUSTUS also has 5 other prokaryotic (bacteria) species. To see if having a GC content more closely resembling the target species makes a difference in the result, rather than the same Domain, I will choose the model bacterial species with the closest GC content and run a test with that as well (as outlined in the table above). For each run, I have run it with the species, -gff3=on as to get it in the correct output format, and --genemodel=intronless as we are doing prokaryotes. Apart from this, nothing other then the assembly file was inputed. 

### Using Bacteria Models

In [ ]:
#for each run, use the table above that we have listed with the bacteria models, assign it to a list
species_model_list = ["s_aureus",
                      "s_aureus",
                      "thermoanaerobacter_tengcongensis",
                      "thermoanaerobacter_tengcongensis",
                      "thermoanaerobacter_tengcongensis",
                      "s_pneumoniae",
                      "s_pneumoniae",
                      "E_coli_K12",
                      "b_pseudomallei",
                      "b_pseudomallei"]

os.mkdir("augustus_bacteria_output") #make an output folder for the augustus bacteria results

for item,species in zip(download_list,species_model_list): #for each organism, and it's associated bacteria species model
    print(f'Running AUGUSTUS for {item}\n') #print what organism we are doing analysis for, this will be useful when we save the terminal output to compare time/computational intensity later.
    os.mkdir(f'augustus_bacteria_output/{item}') #make a folder for it in the augustus bacteria results
    
    path_to_fna = "data/" + item + "/*.fna" #path to where the .fna file exits
    folder_path = "augustus_bacteria_output/" + item + "/" #path to the folder that the output will be in
    #print(path_to_fna)
    output_file = f'augustus_{item}.gff' #name of output file with raw results
    CDS_output_file = f'augustus_{item}_CDS.gff' #name of output file with CDS only results
    
    augustus_command = f'/usr/bin/time --verbose augustus --species={species} --gff3=on --genemodel=intronless {path_to_fna} > {folder_path}{output_file}'
    subprocess.run(augustus_command, shell=True) #command to do augustus using specified species, gff3 format, and intronless model as we are doing prokaryotes.
    #print(augustus_command)
    
    grep_command = f'grep "AUGUSTUS\tCDS" {folder_path}{output_file} > {folder_path}{CDS_output_file}'
    subprocess.run(grep_command, shell=True) #command to get just CDS results
    #print(grep_command)
   
    sed_comman = f"sed -i '1i##gff-version 3' {folder_path}{CDS_output_file}"
    subprocess.run(sed_comman, shell=True) #format it in gff3
    #print(sed_comman)
    print("\n")
    

### Using Archaea Model

In [ ]:
#since there is only one Archaea species, we don't need a list like we did for bacteria

os.mkdir("augustus_archaea_output") #make an output folder for the augustus bacteria results

for item in download_list: #for each organism
    print(f'Running AUGUSTUS for {item}\n') #print what organism we are doing analysis for, this will be useful when we save the terminal output to compare time/computational intensity later.
    os.mkdir(f'augustus_archaea_output/{item}') #make a folder for it in the augustus bacteria results
    
    path_to_fna = "data/" + item + "/*.fna" #path to where the .fna file exits
    folder_path = "augustus_archaea_output/" + item + "/" #path to the folder that the output will be in
    #print(path_to_fna)
    output_file = f'augustus_{item}.gff' #name of output file with raw results
    CDS_output_file = f'augustus_{item}_CDS.gff' #name of output file with CDS only results
    
    augustus_command = f'/usr/bin/time --verbose augustus --species=sulfolobus_solfataricus --gff3=on --genemodel=intronless {path_to_fna} > {folder_path}{output_file}'
    subprocess.run(augustus_command, shell=True) #command to do augustus using specified species, gff3 format, and intronless model as we are doing prokaryotes.
    #print(augustus_command)
    
    grep_command = f'grep "AUGUSTUS\tCDS" {folder_path}{output_file} > {folder_path}{CDS_output_file}'
    subprocess.run(grep_command, shell=True) #command to get just CDS results
    #print(grep_command)
   
    sed_comman = f"sed -i '1i##gff-version 3' {folder_path}{CDS_output_file}"
    subprocess.run(sed_comman, shell=True) #format it in gff3
    #print(sed_comman)
    print("\n")

# Prodigal Analysis


Prodigal requires no model. In running it I just used the default command, specifying the input/output paths as well as the perferred format (gff).

In [ ]:
os.mkdir("prodigal_output") #make an output folder for the augustus bacteria results

for item in download_list: #for each organism
    print(f'Running Prodigal for {item}\n') #print what organism we are doing analysis for, this will be useful when we save the terminal output to compare time/computational intensity later.
    os.mkdir(f'prodigal_output/{item}') #make a folder for it in the prodigal results
    
    path_to_fna = "data/" + item + "/*.fna" #path to where the .fna file exits
    folder_path = "prodigal_output/" + item + "/" #path to the folder that the output will be in
    #print(path_to_fna)
    output_file = f'prodigal_{item}_CDS.gff' #name of output file with results
    
    print(output_file)
    
    prodigal_command = f'/usr/bin/time --verbose prodigal -i {path_to_fna} -o {folder_path}{output_file} -f gff'
    #print(prodigal_command)
    subprocess.run(prodigal_command, shell=True) #command to do prodigal
    
    print("\n")

# FragGeneScan Analysis

* This will have to be done in the directory that you made your FragGeneScan (i.e where the run_FragGeneScan.pl file exist). FragGeneScan also requires no model. In addition to the input and output paths, I also specified that the assembly genomes we are using are complete. 

In [ ]:

os.mkdir("fraggenescan_output") #make an output folder for the augustus bacteria results

for item in download_list: #for each organism
    print(f'Running FragGeneScan for {item}\n') #print what organism we are doing analysis for, this will be useful when we save the terminal output to compare time/computational intensity later.
    os.mkdir(f'fraggenescan_output/{item}') #make a folder for it in the prodigal results
    
    path_to_fna = "../data/" + item + "/*.fna" #path to where the .fna file exits, not full
    
    echo_path = subprocess.run(f"echo {path_to_fna}", shell=True, capture_output=True, text=True) #get full path
    full_path_to_fna = echo_path.stdout.strip() #strip and format
    
    folder_path = "fraggenescan_output/" + item + "/" #path to the folder that the output will be in
    #print(path_to_fna)
    output_file = f'fraggenescan_{item}_CDS' #name of output file with results
    
    print(output_file)
    
    fraggenescan_command = f'/usr/bin/time --verbose ./run_FragGeneScan.pl -genome={full_path_to_fna} -out={folder_path}{output_file} -complete=1 -train=complete'
    
    print(fraggenescan_command)
    subprocess.run(fraggenescan_command, shell=True) #command to do fraggenescan
    
    print("\n")

# Compare Output with RefSeq GFF File Using GFFCompare

In [ ]:
######## AUGUSTUS COMPARISONS - Bacteria ########

os.mkdir("augustus_bacteria_compare") #make an output folder for the augustus bacteria comparison

for item in download_list: #for each organism, and it's associated bacteria species model
    print(f'Running AUGUSTUS bacteria compare for {item}\n') #print what organism we are doing analysis for, this will be useful when we save the terminal output to compare time/computational intensity later.
    os.mkdir(f'augustus_bacteria_compare/{item}') #make a folder for it in the augustus bacteria results
    
    path_to_true_gff = "data/" + item + "/*_CDS.gff" #path to where the .fna file exits
    path_to_test_gff = "augustus_bacteria_output/" + item + "/*_CDS.gff"
    
    folder_path = "augustus_bacteria_compare/" + item + "/" #path to the folder that the output will be in
    #print(path_to_fna)
    
    gff_command = f'/usr/bin/time --verbose gffcompare -r {path_to_true_gff} {path_to_test_gff}'
    subprocess.run(gff_command, shell=True) #command to do comparisons
    print(gff_command)
    
    move_ouput_command = f'mv gffcmp.* {folder_path}'
    subprocess.run(move_ouput_command, shell=True) #command to do move output
    print(move_ouput_command)

    print("\n")
    

######## AUGUSTUS COMPARISONS - Archaea ########

os.mkdir("augustus_archaea_compare") #make an output folder for the augustus archaea comparison

for item in download_list: #for each organism, and it's associated archaea species model
    print(f'Running AUGUSTUS archaea compare for {item}\n') #print what organism we are doing analysis for, this will be useful when we save the terminal output to compare time/computational intensity later.
    os.mkdir(f'augustus_archaea_compare/{item}') #make a folder for it in the augustus archaea results
    
    path_to_true_gff = "data/" + item + "/*_CDS.gff" #path to where the .fna file exits
    path_to_test_gff = "augustus_archaea_output/" + item + "/*_CDS.gff"
    
    folder_path = "augustus_archaea_compare/" + item + "/" #path to the folder that the output will be in
    #print(path_to_fna)
    
    gff_command = f'/usr/bin/time --verbose gffcompare -r {path_to_true_gff} {path_to_test_gff}'
    subprocess.run(gff_command, shell=True) #command to do comparisons
    print(gff_command)
    
    move_ouput_command = f'mv gffcmp.* {folder_path}'
    subprocess.run(move_ouput_command, shell=True) #command to do move output
    print(move_ouput_command)

    print("\n")


######## PRODIGAL COMPARISONS ########

os.mkdir("prodigal_compare") #make an output folder for the Prodigal comparison

for item in download_list: #for each organism
    print(f'Running Prodigal compare for {item}\n') #print what organism we are doing analysis for, this will be useful when we save the terminal output to compare time/computational intensity later.
    os.mkdir(f'prodigal_compare/{item}') #make a folder for it in the Prodigal results
    
    path_to_true_gff = "data/" + item + "/*_CDS.gff" #path to where the .fna file exits
    path_to_test_gff = "prodigal_output/" + item + "/*_CDS.gff"
    
    folder_path = "prodigal_compare/" + item + "/" #path to the folder that the output will be in
    #print(path_to_fna)
    
    gff_command = f'/usr/bin/time --verbose gffcompare -r {path_to_true_gff} {path_to_test_gff}'
    subprocess.run(gff_command, shell=True) #command to do comparisons
    print(gff_command)
    
    move_ouput_command = f'mv gffcmp.* {folder_path}'
    subprocess.run(move_ouput_command, shell=True) #command to do move output
    print(move_ouput_command)

    print("\n")
    

######## FRAGGENESCAN COMPARISONS ########

os.mkdir("fraggenescan_compare") #make an output folder for the fraggenescan comparison

for item in download_list: #for each organism
    print(f'Running FragGeneScan compare for {item}\n') #print what organism we are doing analysis for, this will be useful when we save the terminal output to compare time/computational intensity later.
    os.mkdir(f'fraggenescan_compare/{item}') #make a folder for it in the fraggenescan results
    
    path_to_true_gff = "data/" + item + "/*_CDS.gff" #path to where the .fna file exits
    path_to_test_gff = "fraggenescan_output/" + item + "/*_CDS.gff"
    
    folder_path = "fraggenescan_compare/" + item + "/" #path to the folder that the output will be in
    #print(path_to_fna)
    
    gff_command = f'/usr/bin/time --verbose gffcompare -r {path_to_true_gff} {path_to_test_gff}'
    subprocess.run(gff_command, shell=True) #command to do comparisons
    print(gff_command)
    
    move_ouput_command = f'mv gffcmp.* {folder_path}'
    subprocess.run(move_ouput_command, shell=True) #command to do move output
    print(move_ouput_command)

    print("\n")
    


# Manual Curation of Comparisons Spreadsheet
After all comparisons were done, the output was manually examined to create a spreadsheet that included the base level sensitivity and precision, transcript level sensitivity and precision, locus level sensitivity and precision, number of matching transcripts, matching loci, missed loci, and novel loci for each run. From the time command output, how long each run took (in seconds), as well as the maximum resident set size (KB), was added to the spreadsheet. This information was supplemented by also adding the GC content and base length of each genome. 